In [ ]:
!pip install opendatasets --quiet
!pip install transformers --quiet
!pip install peft==0.5.0 --quiet
!pip install datasets --quiet
!pip install bitsandbytes==0.39.0 --quiet
!pip install accelerate --quiet
!pip install trl --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 18.6 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sun Nov 26 10:24:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    53W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Download

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/fadyelkbeer/arabic-text-summarization-30-000')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: omaratef3221
Your Kaggle Key: ··········


100%|██████████| 30.3M/30.3M [00:02<00:00, 12.3MB/s]


# Imports

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from Data_processing import get_data_final
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from trl import SFTTrainer

import time

# Get Cleaned Data

In [ ]:
data_df = get_data_final()
print(data_df.shape)
data_df.head()

(29201, 2)


,summary,text
0,تناول الفاكهة والخضراوات في موسمها تعرف على أس...,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...
1,فضل خيارات الأطعمة الأرخص ثمنا تباطأ في استهلا...,الأطعمة الصحية ليست باهظة الثمن بالضرورة بل في...
2,ازرع كل ما يمكنك من خضراوات وفاكهة اطه بنفسك و...,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...
3,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع التزم بت...,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...
4,قلل وقت التعرض للشاشات اذهب إلى الطبيب,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...


# Model

In [ ]:
model_path = "inception-mbzuai/jais-13b"

# quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,)

# # bnb_config = BitsAndBytesConfig(
# #     load_in_4bit= True,
# #     bnb_4bit_quant_type= "nf4",
# #     bnb_4bit_compute_dtype= torch.float16,
# #     bnb_4bit_use_double_quant= False,
# # )

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(model_path,
                                             offload_folder="offload",
                                             quantization_config = quantization_config,
                                             trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model), flush=True)


trainable model parameters: 435988520
all model parameters: 6729458360
percentage of trainable model parameters: 6.48%


In [ ]:
for param in model.parameters():
  param.requires_grad = True
print(print_number_of_trainable_model_parameters(model), flush=True)

RuntimeError: ignored

In [ ]:
train=data_df.sample(frac=0.7,random_state=7) # Create training of 70% of the data
test=data_df.drop(train.index) # Create testing by removing the 70% of the train data which will result in 30%

val=test.sample(frac=0.5,random_state=7) # Create validation of 50% of the testing data
test=test.drop(val.index) # Create testing by removing the 50% of the validation data which will result in 50%


print("Training Shape: ", train.shape) # Print the Trainnig shape (rows, columns)
print("Validation Shape: ", val.shape) # Print the Validation shape (rows, columns)
print("Testing Shape: ", test.shape) # Print the Testing shape (rows, columns)

Training Shape:  (20441, 2)
Validation Shape:  (4380, 2)
Testing Shape:  (4380, 2)


In [ ]:
def tokenize_function(example):
    start_prompt = 'قم بتلخيص هذه الفقرة: \n\n'
    end_prompt = '\n\nالتلخيص: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["text"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


# Train Data
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map(tokenize_function, batched=True)
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['summary', 'text',])


# Val Data
val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched=True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['summary', 'text',])


# Test Data
test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched=True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['summary', 'text',])


Map:   0%|          | 0/20441 [00:00<?, ? examples/s]

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

In [ ]:
EPOCHS = 25
LR = 1e-3
BATCH_SIZE = 4

training_output_dir = f'./JAIS_original_training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=training_output_dir,
    learning_rate=LR,
    num_train_epochs=25,
    per_device_train_batch_size=BATCH_SIZE,
    logging_steps=1,
    logging_strategy = 'epoch',
    max_steps=-1,
    # fp16 = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets
)
start = time.time()
trainer.train()
print("Training time for Original model fine tuning: ", round(time.time() - start), " Seconds. ", flush = True)

ValueError: ignored